# SPTAG python wrapper tutorial 

To end-to-end build a vector search online service, it contains two steps:
- Offline build SPTAG index for database vectors
- Online serve the index to support vector search requests from the clients

## Offline build SPTAG index

> Prepare input vectors and metadatas for SPTAG

In [1]:
import os
import numpy as np

vector_number = 100
vector_dimension = 10

# Randomly generate the database vectors. Currently SPTAG only support int8, int16 and float32 data type.
x = np.random.rand(vector_number, vector_dimension).astype(np.float32) 

# Prepare metadata for each vectors, separate them by '\n'. Currently SPTAG python wrapper only support '\n' as the separator
m = ''
for i in range(vector_number):
    m += str(i) + '\n'

> Build SPTAG index for database vectors **x**

In [2]:
import SPTAG

index = SPTAG.AnnIndex('BKT', 'Float', vector_dimension)

# Set the thread number to speed up the build procedure in parallel 
index.SetBuildParam("NumberOfThreads", '4')

# Set the distance type. Currently SPTAG only support Cosine and L2 distances. Here Cosine distance is not the Cosine similarity. The smaller Cosine distance it is, the better.
index.SetBuildParam("DistCalcMethod", 'Cosine') 

if index.BuildWithMetaData(x, m, vector_number, False):
    index.Save("sptag_index") # Save the index to the disk

os.listdir('sptag_index')

['deletes.bin',
 'graph.bin',
 'indexloader.ini',
 'metadata.bin',
 'metadataIndex.bin',
 'tree.bin',
 'vectors.bin']

In [3]:
# Local index test on the vector search
index = SPTAG.AnnIndex.Load('sptag_index')

# prepare query vector
q = np.random.rand(vector_dimension).astype(np.float32)

result = index.SearchWithMetaData(q, 3) # Search k=3 nearest vectors for query vector q
print (result[0]) # nearest k vector ids
print (result[1]) # nearest k vector distances
print (result[2]) # nearest k vector metadatas

[47, 10, 92]
[-1.0077600479125977, -0.9659097194671631, -0.9412962198257446]
['47\n', '10\n', '92\n']


## Online serve the index

Start the vector search service on the host machine which listens for the client requests on the port 8000

> Write a server configuration file **service.ini** as follows:

```bash
[Service]
ListenAddr=0.0.0.0
ListenPort=8000
ThreadNumber=8
SocketThreadNumber=8

[QueryConfig]
DefaultMaxResultNumber=6
DefaultSeparator=|

[Index]
List=MyIndex

[Index_MyIndex]
IndexFolder=sptag_index
```

> Start the server on the host machine

```bash
Server.exe -m socket -c service.ini
```

It will print the follow messages:

```bash
Setting TreeFilePath with value tree.bin
Setting GraphFilePath with value graph.bin
Setting VectorFilePath with value vectors.bin
Setting DeleteVectorFilePath with value deletes.bin
Setting BKTNumber with value 1
Setting BKTKmeansK with value 32
Setting BKTLeafSize with value 8
Setting Samples with value 1000
Setting TPTNumber with value 32
Setting TPTLeafSize with value 2000
Setting NumTopDimensionTpTreeSplit with value 5
Setting NeighborhoodSize with value 32
Setting GraphNeighborhoodScale with value 2
Setting GraphCEFScale with value 2
Setting RefineIterations with value 2
Setting CEF with value 1000
Setting MaxCheckForRefineGraph with value 8192
Setting NumberOfThreads with value 4
Setting DistCalcMethod with value Cosine
Setting DeletePercentageForRefine with value 0.400000
Setting AddCountForRebuild with value 1000
Setting MaxCheck with value 8192
Setting ThresholdOfNumberOfContinuousNoBetterPropagation with value 3
Setting NumberOfInitialDynamicPivots with value 50
Setting NumberOfOtherDynamicPivots with value 4
Load Vector From sptag_index\vectors.bin
Load Vector (100, 10) Finish!
Load BKT From sptag_index\tree.bin
Load BKT (1,101) Finish!
Load Graph From sptag_index\graph.bin
Load Graph (100, 32) Finish!
Load DeleteID From sptag_index\deletes.bin
Load DeleteID (100, 1) Finish!
Start to listen 0.0.0.0:8000 ...
```

> Start python client to connect to the server and send vector search request.

In [4]:
import SPTAGClient
import time

# connect to the server
client = SPTAGClient.AnnClient('127.0.0.1', '8000')
while not client.IsConnected():
    time.sleep(1)
client.SetTimeoutMilliseconds(18000)

k = 3
vector_dimension = 10
# prepare query vector
q = np.random.rand(vector_dimension).astype(np.float32)

result = client.Search(q, k, 'Float', True) # AnnClient.Search(query_vector, knn, data_type, with_metadata)

print (result[0]) # nearest k vector ids
print (result[1]) # nearest k vector distances
print (result[2]) # nearest k vector metadatas



[15, 35, 75]
[-1.094923973083496, -1.0913333892822266, -1.0795197486877441]
['15\n', '35\n', '75\n']
